In [1]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import random
import keras
import time
import os

from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from sklearn import metrics
from random import randint

from scipy.io import loadmat
from keras.models import Sequential
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
w, h = model_wh('432x368')
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))

[2019-06-03 16:37:44,851] [TfPoseEstimator] [INFO] loading graph from C:\Users\Eilin\JackChen\Final_project\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)
2019-06-03 16:37:44,851 INFO loading graph from C:\Users\Eilin\JackChen\Final_project\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


In [3]:
model = load_model('fall_floor_lstm_v3.h5')#lstm

In [4]:
cap = cv2.VideoCapture('./videos/test1.avi')

In [5]:
frame_cnt = 0
frame_idx = 0
#最多就取32幀
action_history = []

In [6]:
cap.set(cv2.CAP_PROP_POS_MSEC, (frame_cnt*85))#可擷取到32個frames
ret_val, image = cap.read()
while ret_val:
    if frame_idx > 31:
        action_predict = np.zeros([1, 32, 36])
        start_idx = frame_idx - 32
        for i in range(32):
            action_predict[0, i, :] = action_history[start_idx+i]
        result = model.predict(action_predict)
        print('Predict:', result)
        c = np.argmax(result)
        if c < 2:
            print('State: Safe')
        else:
            print('State: Dangerous')
        frame_cnt = 0
    
    humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
    action = np.zeros([36,])
    if humans:
        width, height, channel = image.shape
        for part_idx in humans[0].body_parts:
            action[part_idx*2] = humans[0].body_parts[part_idx].x*width + 0.5
            action[part_idx*2+1] = humans[0].body_parts[part_idx].y*height + 0.5
    action_history.append(action)
        
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    ret_val, image = cap.read()
    print("The number", frame_cnt, "of frame")
    print('Read a new frame: ', ret_val)
    frame_cnt += 1
    frame_idx += 1
    
cap.release()

The number 0 of frame
Read a new frame:  True
The number 1 of frame
Read a new frame:  True
The number 2 of frame
Read a new frame:  True
The number 3 of frame
Read a new frame:  True
The number 4 of frame
Read a new frame:  True
The number 5 of frame
Read a new frame:  True
The number 6 of frame
Read a new frame:  True
The number 7 of frame
Read a new frame:  True
The number 8 of frame
Read a new frame:  True
The number 9 of frame
Read a new frame:  True
The number 10 of frame
Read a new frame:  True
The number 11 of frame
Read a new frame:  True
The number 12 of frame
Read a new frame:  True
The number 13 of frame
Read a new frame:  True
The number 14 of frame
Read a new frame:  True
The number 15 of frame
Read a new frame:  True
The number 16 of frame
Read a new frame:  True
The number 17 of frame
Read a new frame:  True
The number 18 of frame
Read a new frame:  True
The number 19 of frame
Read a new frame:  True
The number 20 of frame
Read a new frame:  True
The number 21 of frame
